# Prvi del

V priponki prilagam .csv datoteko, v kateri je precej stolpcev. V neki točki se je v bazi spremenila nomenklatura vrednosti, in sicer je sprememba v stilu 'V1h1' in V1h01', za vse oznake do vključno 9.

Logično, se do neke točke zapisujejo vrednosti v eno, nato v drugo spremenljivko, posledično je vedno ena izmed spremenljivk NaN.
Rad bi združil celotno tabelo v eno obliko, na način, da se veljavne vrednosti iz stolpcev V1hX prepišejo na neveljavne vrednosti stolpcev V1h0X.

Sedaj sem iz tabele izvlekel samo te stolpce, izvedel update in po združevanju vpisal nazaj v prvi dataframe:

Ali je morda mogoče omenjeno narediti hitreje, z manj vrsticami torej.

In [23]:
import pandas as pd
import numpy as np

In [24]:
piv = pd.read_csv("../data/test_piv.csv", index_col=0)

In [25]:
piv.head(3)

,Timestamp,MrpIdentification,U1h01,U1h02,U1h03,U1h04,U1h05,U1h06,U1h07,U1h08,...,U1h23,U1h24,U1h25,U1h3,U1h4,U1h5,U1h6,U1h7,U1h8,U1h9
0,2018-01-01 00:00:00,RTP S2,69021.91,30.21,204.77,10.07,83.92,0.0,302.12,3.36,...,117.49,0.0,97.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:10:00,RTP S2,68807.07,30.21,208.13,10.07,70.50,0.0,298.77,3.36,...,134.28,0.0,97.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:20:00,RTP S2,68692.93,30.21,204.77,10.07,83.92,0.0,302.12,6.71,...,127.56,0.0,97.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
import re

zero_padded_column_names = [name for name in piv.columns if re.search(r"U1h0\d+", name)]
valid_values_column_names = [name.replace("0", "") for name in zero_padded_column_names]

print(f"Zero padded columns: {zero_padded_column_names}")
print(f"Following columns will be merged with zero padded columns: {valid_values_column_names}")

Zero padded columns: ['U1h01', 'U1h02', 'U1h03', 'U1h04', 'U1h05', 'U1h06', 'U1h07', 'U1h08', 'U1h09']
Following columns will be merged with zero padded columns: ['U1h1', 'U1h2', 'U1h3', 'U1h4', 'U1h5', 'U1h6', 'U1h7', 'U1h8', 'U1h9']


In [27]:
column_mapping = {old_col: new_col for (old_col, new_col) in zip(valid_values_column_names, zero_padded_column_names)}
column_mapping

{'U1h1': 'U1h01',
 'U1h2': 'U1h02',
 'U1h3': 'U1h03',
 'U1h4': 'U1h04',
 'U1h5': 'U1h05',
 'U1h6': 'U1h06',
 'U1h7': 'U1h07',
 'U1h8': 'U1h08',
 'U1h9': 'U1h09'}

In [10]:
test_piv_x = piv.loc[:,valid_values_column_names].copy()
test_piv_x.rename(columns=column_mapping, inplace=True)
print(test_piv_x.columns)

Index(['U1h01', 'U1h02', 'U1h03', 'U1h04', 'U1h05', 'U1h06', 'U1h07', 'U1h08',
       'U1h09'],
      dtype='object')


In [28]:
piv.update(test_piv_x, overwrite=False, errors="raise")
piv.drop(valid_values_column_names, axis="columns", inplace=True)

In [29]:
piv.head(2)

,Timestamp,MrpIdentification,U1h01,U1h02,U1h03,U1h04,U1h05,U1h06,U1h07,U1h08,...,U1h16,U1h17,U1h18,U1h19,U1h20,U1h21,U1h22,U1h23,U1h24,U1h25
0,2018-01-01 00:00:00,RTP S2,69021.91,30.21,204.77,10.07,83.92,0.0,302.12,3.36,...,3.36,26.86,0.0,3.36,0.0,3.36,3.36,117.49,0.0,97.35
1,2018-01-01 00:10:00,RTP S2,68807.07,30.21,208.13,10.07,70.50,0.0,298.77,3.36,...,3.36,20.14,0.0,10.07,0.0,3.36,3.36,134.28,0.0,97.35


In [30]:
piv.tail(2)

,Timestamp,MrpIdentification,U1h01,U1h02,U1h03,U1h04,U1h05,U1h06,U1h07,U1h08,...,U1h16,U1h17,U1h18,U1h19,U1h20,U1h21,U1h22,U1h23,U1h24,U1h25
210382,2021-12-31 23:40:00,RTP S2,68988.34,0.0,90.64,3.36,466.61,0.0,325.62,3.36,...,0.0,57.07,0.0,13.43,0.0,3.36,0.0,13.43,3.36,20.14
210383,2021-12-31 23:50:00,RTP S2,68797.00,0.0,90.64,3.36,453.19,0.0,318.91,3.36,...,0.0,60.42,0.0,10.07,0.0,3.36,0.0,16.78,3.36,13.43


Ali je možno spodnje izvesti z nekim dinamičnim ukazom? Trenutno sem hardcodal dict za preimenovanje, vendar bi bilo to v primeru večjega nabora spremenljivk nekoliko neugodno:

Izvesti celoten nabor zeropadded afer 'h':

    ['U1h1',
    'U1h10',
    'U1h11',
    'U1h12',
    'U1h13',
    'U1h14',
    'U1h15',
    'U1h16',
    'U1h17',
    'U1h18',
    'U1h19',
    'U1h2',
    'U1h20',
    'U1h21',
    'U1h22',
    'U1h23',
    'U1h24',
    'U1h25',
    'U1h3',
    'U1h4',
    'U1h5',
    'U1h6',
    'U1h7',
    'U1h8',
    'U1h9']


In [ ]:
# več zgoraj, s pomočjo regexa

## Matematične operacije NotNull

In [31]:
a = pd.DataFrame({"a": [1,2,3, np.nan, 4.5], "b": [np.nan, np.nan, 5,6,7.6]})

In [32]:
a

,a,b
0,1.0,NaN
1,2.0,NaN
2,3.0,5.0
3,NaN,6.0
4,4.5,7.6


In [33]:
a.isna().sum()

a    1
b    2
dtype: int64

In [34]:
a.div(4, axis="rows")

,a,b
0,0.250,NaN
1,0.500,NaN
2,0.750,1.25
3,NaN,1.50
4,1.125,1.90


In [35]:
a.div(4, axis="rows").apply(np.square)

,a,b
0,0.062500,NaN
1,0.250000,NaN
2,0.562500,1.5625
3,NaN,2.2500
4,1.265625,3.6100


In [37]:
a.div(4, axis="rows").apply(np.square).apply(np.sum, axis="columns")

0    0.062500
1    0.250000
2    2.125000
3    2.250000
4    4.875625
dtype: float64

In [38]:
a.div(4, axis="rows").apply(np.square).apply(np.sum, axis="columns").apply(np.sqrt)*100

0     25.000000
1     50.000000
2    145.773797
3    150.000000
4    220.808175
dtype: float64

In [40]:
# boljši način
# pow - but with support to substitute a fill_value for missing data in one of the inputs
a.div(4, axis="rows").pow(2).sum(axis="columns").pow(1./2)*100

0     25.000000
1     50.000000
2    145.773797
3    150.000000
4    220.808175
dtype: float64

In [41]:
a.div(4, axis="rows").pow(2).sum(axis="columns", skipna=False).pow(1./2)*100

0           NaN
1           NaN
2    145.773797
3           NaN
4    220.808175
dtype: float64

### Time diff

In [42]:
a = pd.DataFrame({"a": range(1,100_000), "b": range(1, 100_000)})
a["b"] = a["b"]*2
a.head()

,a,b
0,1,2
1,2,4
2,3,6
3,4,8
4,5,10


In [43]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       99999 non-null  int64
 1   b       99999 non-null  int64
dtypes: int64(2)
memory usage: 1.5 MB


In [44]:
%%timeit
a.div(4, axis="rows").apply(np.square).apply(np.sum, axis="columns").apply(np.sqrt)*100

7.3 s ± 76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit
a.div(4, axis="rows").pow(2).sum(axis="columns", skipna=False).pow(1./2)*100

5.01 ms ± 569 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
